# Huggingface POC 🚀🚀🚀


En este POC voy a enfocarme en realizar clasificación binaria. Para esto, voy a escoger el label con mayor frequencia de la columna `tipo_de_evento`. 

La clasificación va a estar enfocada en clasificar: 
   - Denuncia Falta de Servicio (label = 1)
   - No es Denuncia Falta de Servicio (label = 2)


|                                        |   tipo_de_evento |
|:---------------------------------------|-----------------:|
| DENUNCIA FALTA DEL SERVICIO            |             1134 |
| FALTA DE SERVICIO                      |              525 |
| PROTESTA FALTA DEL SERVICIO            |              324 |
| DENUNCIA SERVICIO MALA CALIDAD         |              146 |
| DENUNCIA ALTO COSTO SERVICIO           |               96 |
| SERVICIO MALA CALIDAD                  |               69 |
| DENUNCIA COBRO EN DIVISAS              |               37 |
| ALTO COSTO SERVICIO                    |               21 |
| COBRO EN DIVISAS                       |               20 |
| PROTESTA ALTO COSTO SERVICIO           |                6 |
| DENUNCIA                               |                5 |
| PROTESTA                               |                4 |
| ENEFERMEDAD ASOCIADA                   |                3 |
| PROTESTA COBRO EN DIVISAS              |                3 |
| METODO ALTERNATIVO PARA COCINAR        |                2 |
| DENUNCIA TIEMPO PARA ADQUIRIR SERVICIO |                2 |
| ENFERMEDAD ASOCIADA                    |                1 |
| DENUNCIA RETARDO EN LA ENTREGA         |                1 |
| PROTESTA RETARDO EN LA ENTREGA         |                1 |
| PROTESTA TIEMPO PARA ADQUIRIR SERVICIO |                1 |

# Installing Missing Libraries & Imports

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

## Imports 

In [ ]:
import pandas as pd
import numpy as np
from transformers import RobertaModel, RobertaTokenizer, Trainer, TrainingArguments, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Reading dataset

In [ ]:
df_elpitazo_pscdd = pd.read_csv("/gdrive/MyDrive/sambil/datasets/elpitazo_positivelabels_devdataset.csv")

## Data Wrangling


In [ ]:
# print(df_elpitazo_pscdd.tipo_de_evento.value_counts().to_markdown())

In [ ]:
df_elpitazo_pscdd["label"] = (df_elpitazo_pscdd.tipo_de_evento == "DENUNCIA FALTA DEL SERVICIO").astype(int)
df_elpitazo_pscdd = df_elpitazo_pscdd.convert_dtypes()

In [ ]:
df_denuncia_texto = df_elpitazo_pscdd[["label","text"]]

# Preparing to Model 🤓



In [ ]:
model = RobertaForSequenceClassification.from_pretrained("mrm8488/RuPERTa-base", num_labels=1)
tokenizer = RobertaTokenizer.from_pretrained('mrm8488/RuPERTa-base')

In [ ]:
# df_denuncia_texto.loc[:,"id"] = list(range(df_denuncia_texto.shape[0]))

# Este paso es muy importante. TODAS las columnas deben ser de tipo String
df_denuncia_texto = df_denuncia_texto.astype(str)
train_df = df_denuncia_texto.sample(frac = 0.8, random_state= 212)
eval_df = df_denuncia_texto[~df_denuncia_texto.index.isin(train_df.index)]

In [ ]:
# train_texts, val_texts, train_labels, val_labels = train_test_split(df_denuncia_texto["text"].to_list(), 
#                                                                     df_denuncia_texto["label_denuncia_falta_servicio"].to_list(), test_size=.2)


train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

In [ ]:
# method to tokenize the data
def tokenize(batch):
    return tokenizer(batch['text'], batch['label'], padding=True, truncation=True, max_length=512)

In [ ]:
# training and test data definition and set format
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
eval_dataset = eval_dataset.map(tokenize, batched=True, batch_size=len(eval_dataset))

train_dataset = train_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
eval_dataset = eval_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
eval_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])


# Training Model 🏋️‍♀️



In [ ]:
!pwd

In [ ]:
training_args = TrainingArguments(
    output_dir='/gdrive/MyDrive/sambil/poc_transformers/logs',          # output directory
    num_train_epochs=50,             # total # of training epochs
    per_device_train_batch_size=10,  # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/gdrive/MyDrive/sambil/poc_transformers/results',            # directory for storing logs
)

# instantiate the pytorch trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,           # evaluation dataset
)

Estoy Bloqueado en este paso

Output:

```
TypeError                                 Traceback (most recent call last)
<ipython-input-26-1a10a9109f73> in <module>()
      1 # training with custom dataset
----> 2 trainer.train()

15 frames
/usr/local/lib/python3.7/dist-packages/datasets/formatting/torch_formatter.py in _tensorize(self, value)
     42             default_dtype = {"dtype": torch.float32}
     43 
---> 44         return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})
     45 
     46     def _recursive_tensorize(self, data_struct: dict):

TypeError: new(): invalid data type 'numpy.str_'
```

In [ ]:
# training with custom dataset
# Estoy bloqueado en este paso
trainer.train()